In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

<IPython.core.display.Javascript object>

In [3]:
def one_hot_encode(X, encode_cols, index=None):
    other_cols = [c for c in X.columns if c not in encode_cols]

    if index is None:
        index = X.index

    ct = ColumnTransformer(
        #   Format
        #   [("name of step", what_to_do(), [what columns to do it to])]
        [("one hot encode", OneHotEncoder(drop="first", sparse=False), encode_cols)],
        remainder="passthrough",
    )

    ct.fit(X)

    # This is not the flexible and definitely not the most
    # readable way to get column names, a function would be better
    encoded_names = ct.transformers_[0][1].get_feature_names()
    encoded_names = list(encoded_names)

    X_encoded = ct.transform(X)
    X_encoded = pd.DataFrame(X_encoded, columns=encoded_names + other_cols, index=index)

    for i, name in enumerate(encode_cols):
        X_encoded.columns = X_encoded.columns.str.replace(f"^x{i}", name)

    return X_encoded

<IPython.core.display.Javascript object>

# 🎄🌳🌴🌱🌲

☝️That's a pretty random forest

We're going to revisit the mammographic mass data set.  Details below.

Dataset from UCI can be found [here](http://archive.ics.uci.edu/ml/datasets/mammographic+mass).

1. BI-RADS assessment: 1 to 5 (ordinal)
2. Age: patient's age in years (integer)
3. Shape: mass shape: round=1 oval=2 lobular=3 irregular=4 (nominal)
4. Margin: mass margin: circumscribed=1 microlobulated=2 obscured=3 ill-defined=4 spiculated=5 (nominal)
5. Density: mass density high=1 iso=2 low=3 fat-containing=4 (ordinal)
6. Severity: benign=0 or malignant=1 (binary)

## Data prep time!

In [4]:
breast_cancer = pd.read_csv(data_url, names=columns)

(961, 6)

<IPython.core.display.Javascript object>

Do some things to get to know your data.

We see in the `head()` output some `?` in the `Density` column.  This might be the cause for why every column is an object rather than numeric.  How can we investigate if `?` is the only cause of our columns being object type?  We want to make sure we won't drop out anything useful.

We'll just remove any `np.nan`s.

Okie doke, from the description we had some 'nominal' (aka categorical columns).  We want to one-hot encode these.  The data description was a long way up.  So I'll just let you know the nominal columns are: `['Shape', 'Margin']`.

In [9]:
breast_cancer = one_hot_encode(breast_cancer, ["Shape", "Margin"])
breast_cancer.head(3)

,Shape_2.0,Shape_3.0,Shape_4.0,Margin_2.0,Margin_3.0,Margin_4.0,Margin_5.0,BI-RADS,Age,Density,Severity
0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0,67.0,3.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,5.0,58.0,3.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,28.0,3.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,57.0,3.0,1.0
10,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.0,76.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
956,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,47.0,3.0,0.0
957,0.0,0.0,1.0,0.0,0.0,0.0,1.0,4.0,56.0,3.0,1.0
958,0.0,0.0,1.0,0.0,0.0,0.0,1.0,4.0,64.0,3.0,0.0
959,0.0,0.0,1.0,0.0,0.0,0.0,1.0,5.0,66.0,3.0,1.0


<IPython.core.display.Javascript object>

Last bit of data prep is to separate out into our `X` and `y` components and `train_test_split()`.  We're predicting the `'Severity'` variable.

In [11]:
X = breast_cancer.drop(columns=["Severity"])
y = breast_cancer["Severity"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

<IPython.core.display.Javascript object>

## Random Forest background
### Concept 1: Bootstrapping ☠️

Fancier name than method.  Bootstrapping is repeatedly sampling with replacement.

In [16]:
# X_train.sample?

<IPython.core.display.Javascript object>

Sample 3 rows from `X_train`.

Select the same 3 rows from `y_train`

Let's write a function to do this for us.

In [ ]:
def xy_sample(X, y, size, random_state=None):
    # Do stuff
    return X_sample, y_sample

So all we want to do is repeat that a few times.

In [ ]:
n_samples = 5
sample_size = 3

bootstrap_samples = []
# Fill in the for loop for us to iterate and make samples
# The number of samples we want to make is stored in n_samples
for ____ in _____:
    # Perform the sampling like we just did
    # Use the sample_size variable
    _____
    
    # Store in a dictionary to have nice X y labels
    train_sample = {
        'X': X_sample,
        'y': y_sample
    }
    
    # Store all our samples together in a list
    bootstrap_samples.____(train_sample)


bootstrap_samples

Boom 💥we're bonified bootstrappers.

### Concept 2: Bagging 💰

Kind of some overlap with concept 1....

<font color='red'>B</font><font color='blue'>AGGING</font> = <font color='red'>B</font>ootstrap <font color='blue'>AGG</font>regat<font color='blue'>ING</font>

* Step 1: Build a bunch of models on bootstrap samples
* Step 2: Aggregate the predictions of each model
* Step 3: dQw4w9WgXcQ
* Step 4: Profit

In [ ]:
# Create a sample of size 10 like we've been doing


# Fit a decision tree to this sample
tree_1 = DecisionTreeClassifier()
tree_1.fit(X_sample, y_sample)

Second verse, same as the first.

In [ ]:
# Create a sample of size 10 like we've been doing


# Fit a decision tree to this sample
tree_2 = DecisionTreeClassifier()
tree_2.fit(X_sample, y_sample)

Again!

In [ ]:
# Create a sample of size 10 like we've been doing


# Fit a decision tree to this sample
tree_3 = DecisionTreeClassifier()
tree_3.fit(X_sample, y_sample)

In [ ]:
pred_1 = tree_1.predict(X_test)
pred_2 = tree_2.predict(X_test)
pred_3 = tree_3.predict(X_test)
pred_df = pd.DataFrame({'pred_1': pred_1, 'pred_2': pred_2, 'pred_3': pred_3})
pred_df

Who do we believe??  Let's be fair and just rulers, we'll take all our trees' votes into consideration like a true democracy.

In [ ]:
pred_df['avg_vote'] = ____
pred_df

Convert the `'avg_vote'` column to a binary label.  Use 0.5 as a cutoff

What Percentage of the predictions are correct?

We just fit 3 pretty naive models.  I say naive because they each only saw 10 records, but there's strength in numbers! This is the idea behind bagging, each model sees a different side of the data so they have different 'experiences' and 'perspectives' on whats right and wrong.  By considering all of the 'opinions' equally we avoid overfitting and we're able to get higher accuracy (in general) than using a single model.

Here comes the downside...

When we did just 1 decision tree, we were able to plot a nice diagram of how it made its decisions.  In our example we just made 3 trees, we could plot each one, but trying to view all these decisions would be a lot.  So we just lost the nice intrepretability that came with a single tree.  In practice, we'll typically have more than 3 trees and this becomes harder and harder to explain (we'll see a way to combat this).

### Concept 3: Random subspace 🌒

Our `X` component is sometimes referred to as our 'feature space'.  A 'subspace' is a subset of a 'space'.  So this fancy term just means that we'll be taking a sample of our columns.  We do this without replacement.

In [ ]:
# X_train.sample?

Well that wasn't too bad, but how does it fit into a random forest?  A random forest will only look at a few of the columns for each decision (i.e. a random subspace).  By doing this, we further protect against overfitting.  It's assuming that we want to learn patterns from every one of our features, if we happened to have a really powerful feature, we might end up only learning from it.  But with a random subspace, that powerful feature won't always be there as a crutch and so we're forced to learn from our other columns too.

So we just defined all the concepts of a random forest. Let's use one.

## Random Forests in action

Fit a random forest classifier to the data.

In [ ]:
model = _____
model.fit(X_train, y_train)

Print out the accuracy of the predictor on the training and test data.

In [ ]:
train_score = model.____
test_score = model.____

print(f'train_score: ____')
print(f'test_score: ____')

Let's see more than just accuracy, how can we see a view of our true-positives, false-positives, etc.?

Based on this output, do we have higher precision or recall?  What `sklearn` function could we use to prove this?

### Importance for intepretability

The 'importances' are stored in the `feature_importances_` attribute of our model.  What does the trailing underscore mean?

Store the importances in a dataframe with a column for each features name.

Order the dataframe from most to least important.

So shape is the most important feature in determining if these mammographic masses are benign or malignant.  What does that mean?  Remember that each feature is only chosen if it's the best split available, and that the way this is chosen is based on the 'information gain'.  We have a lot of trees, and we aggregate these measures of information gain across all the trees to get importance.  So the more important a feature, the more useful it was in separating our 2 classes across all of our forest.